# 2.1.1 Zeichenerklärung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI
import asyncio

instructions = "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()

In [18]:

# Zeichenerklärung – WITH BOUNDING BOX
# SKIP: ze_tt_raw_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-bb-crop-raw.png"
ze_tt_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-bb-crop.png"
ze_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-bb-crop.png"

async def run(ze_path):
    base64_ze, img_size = parser.imageToBase64(ze_path)
    print("image_size:", img_size)
    msg = jarvis.request([
        {
            "type": "text",
            "text": 'Gebe die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform aus.'
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_ze}",
                "detail": "high"
            }
        },
    ])
    return msg

messages = await asyncio.gather(run(ze_tt_path), run(ze_path))
for msg in messages:
    print("#############################################")
    print(msg)

image_size: [768, 1117]
image_size: [728, 2048]
Hier ist die rot umrandete Tabellenerklärung in ASCII-Tabellenform:

```
+-------------------+----------------------+
| Baugebiet         | Zahl der Vollgeschosse|
+-------------------+----------------------+
| Grundflächenzahl  | Geschossflächenzahl  |
+-------------------+----------------------+
| Bauweise          |                      |
+-------------------+----------------------+
| Dachform und      |                      |
| Dachneigung       |                      |
+-------------------+----------------------+
```
Hier ist die rot umrandete Tabellenerklärung in ASCII-Tabellenform:

```
+-------------------+-----------------------+
| Baugebiet         | Zahl der Vollgeschosse|
+-------------------+-----------------------+
| Grundflächenzahl  | Geschossflächenzahl   |
+-------------------+-----------------------+
| -                 | Bauweise              |
+-------------------+-----------------------+
| Dachform und      |        

In [43]:
# Zeichenerklärung – Komplette Definitionen Nutzungsschablone
ze_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-bb-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

jarvis.addContext([
    {
        "type": "text",
        "text": f'ASCII-Nutzungsschablone: {messages[1]}'
    }
])

msg = jarvis.request([
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
    {
        "type": "text",
        "text": 'Fasse alle relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone aus der Zeichenerklärung zusammen. Ausgabe im JSON-Format: {<Parameter>: {"Definition": "", "Beispiel": {<Zeichen>: <Wert>}}}'
    },
])

print(msg)
jarvis.clearContext()

image_size: [728, 2048]
```json
{
  "Baugebiet": {
    "Definition": "Bauland und Art der Nutzung (§ 9 (1) 1 BauGB)",
    "Beispiel": {
      "WR": "Reines Wohngebiet (§ 3 BauNVO)",
      "WA": "Allgemeines Wohngebiet (§ 4 BauNVO)",
      "MI": "Mischgebiet (§ 6 BauNVO)",
      "GE2": "Eingeschränktes Gewerbegebiet für nicht wesentlich störende Gewerbebetriebe (§ 8 BauNVO)"
    }
  },
  "Zahl der Vollgeschosse": {
    "Definition": "Zahl der Vollgeschosse (§ 18 BauNVO)",
    "Beispiel": {
      "9": "9 Vollgeschosse"
    }
  },
  "Grundflächenzahl": {
    "Definition": "Grundflächenzahl (GRZ) (§ 19 BauNVO)",
    "Beispiel": {
      "0,4": "GRZ 0,4",
      "0,3": "GRZ 0,3"
    }
  },
  "Geschossflächenzahl": {
    "Definition": "Geschossflächenzahl (GFZ) (§ 20 BauNVO)",
    "Beispiel": {
      "0,5": "GFZ 0,5"
    }
  },
  "Bauweise": {
    "Definition": "Bauweise (§ 1 (1) 1b BauGB und § 22 BauNVO)",
    "Beispiel": {
      "I": "offene Bauweise, nur Einzel- und Doppelhäuser zulässig",
